# Use Numerov to solve the PIB in 1D.
In this example, we use atomic units. Then $\hbar=1$. Mass unit is electron mass $m_e$=1. Energy unit is Hartree $E_h$=1. Length unit is bohr.

Particle mass $m=1$ ($m_e$), box length L=1 (bhor), $x\in [0,1]$
$$-\frac{\hbar^2}{2m}\psi''=E\psi$$

$$\psi(0)=0$$

$$\psi(1)=0$$

**For simplicity, assume we already know the energy of the first eigen state is $\frac{n^2h^2}{8mL^2}=\frac{(2\pi\hbar)^2n^2}{8mL^2}=\frac{\pi^2}{2}$ (bohr). Let's use Numerov to solve the wave function for the first eigen state.**

The ODE can be simplified as

$$ \psi''=-\pi^2 \psi$$

Based on Numerov method, Schrodinger equation of the following form

$$\psi''=-k^2(x) \psi$$

has the finite difference approximation

$$\psi(x_{n+1}) = 2\psi(x_n)-\psi(x_{n-1})-\delta^2k^2(x_n)\psi(x_n)$$

and in this case $k^2(x_n)=\pi^2$

Writing the code is straightforwad, except for one immediate problem:

**Q:** Whether we start the propogation from the left or the right boundary, we only know $\psi(x_0)$, but the propogation needs two known points $\psi(x_0)$ and $\psi(x_1)$ to get the 3rd point $\psi(x_2)$.

## Run the following code to explore how the grid size can influence the result

* Drag the slider bar to adjust the grid size (how many discrete points to use in the range [0,1])
* Then answer the questions below
    1. Do the numerical solutions obtained from propogating from left and from right match with each other?
    1. Have you noticed that the x axis scale is changing? It's because the psi value at the send initial point is guessed, and the numerical solution is not normalized. How can we handle that?
    1. How does the runtime change with grid size?

In [1]:
%matplotlib widget
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt 
import time

class NumerovSolverPIB:
    def __init__(self, xlower, xupper, npoints=1000):
        self.xlower = xlower
        self.xupper = xupper
        self.npoints = npoints
        self.x = np.linspace(self.xlower,self.xupper,self.npoints)
        self.delta = self.x[1]-self.x[0]
        self.x1 = 1e-2 # A small positive value as the initial guess the psi value at the second grid point
        self.psi_left = None
        self.psi_right = None
        self.k2 = np.pi**2
        self.prob_left = None
        self.prob_right = None
    
    def propogateNumerov(self,x0,x1,psi0,psi1):
        psi2 = 2*psi1 - psi0 - self.k2*psi1*self.delta**2
        return psi2
    
    def Numerov_left(self):
        self.psi_left = np.zeros(len(self.x))
        self.psi_left[1] = self.x1
        for i in range(1,len(self.x)-1):
            self.psi_left[i+1] = self.propogateNumerov(self.x[i-1],self.x[i],self.psi_left[i-1],self.psi_left[i])

    def Numerov_right(self):
        self.psi_right = np.zeros(len(self.x))
        self.psi_right[-2] = self.x1
        for i in range(len(self.x)-2,0,-1):
            self.psi_right[i-1] = self.propogateNumerov(self.x[i+1],self.x[i],self.psi_right[i+1],self.psi_right[i])
        
    

In [2]:
# Number of discrete points to use in the range
n_options = widgets.IntSlider(
    value=10,
    min=3,
    max=101,
    step=5,
    description=r'npoints:',
    disabled=False,
)


In [3]:
fig1=plt.figure()
@widgets.interact(npoints=n_options)
def update(npoints):
    plt.clf()
    solver=NumerovSolverPIB(0,1,npoints)
    start=time.time()
    solver.Numerov_left()
    solver.Numerov_right()
    end=time.time()
    timetaken=end-start
    
    # The wave funciton is not normalized. 
    prob = np.trapz(np.power(solver.psi_left,2),solver.x)
    plt.title("Numerov solution to PIB\n" + "Time taken: {:.2f} ms".format(timetaken*1000))
    
    # plot both the lines and points to make it visually better
    plt.plot(solver.x, solver.psi_left, c='b',label=r'$\psi_\mathrm{left}$')
    plt.scatter(solver.x, solver.psi_left, c='b')
    
    plt.plot(solver.x, solver.psi_right, c='r',label=r'$\psi_\mathrm{right}$')
    plt.scatter(solver.x, solver.psi_right, c='r')
    plt.xlabel(r'$x$ (bohr)')
    plt.ylabel(r'$\psi(x)$')
    plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=10, description='npoints:', max=101, min=3, step=5), Output()), _dom_cla…

## Let's investigate the integral of the probability density and see how to normalize the wave function
* Make a guess: are the integrals of psi_left and psi_right the same?

In [4]:
fig2=plt.figure()
@widgets.interact(npoints=n_options)
def update2(npoints):
    plt.clf()
    solver=NumerovSolverPIB(0,1,npoints)
    start=time.time()
    solver.Numerov_left()
    solver.Numerov_right()
    end=time.time()
    timetaken=end-start
    
    # Let's calculate the numerical integral of psi**2
    prob_left = np.trapz(np.power(solver.psi_left,2),solver.x)
    prob_right = np.trapz(np.power(solver.psi_right,2),solver.x)
    plt.title("Numerov solution to PIB, "
              + r"$\int\psi_\mathrm{left}$=" + "{:.2g}".format(prob_left)
              + r", $\int\psi_\mathrm{right}=$" + "{:.2g}".format(prob_right)
              + "\nTime taken: {:.2f} ms".format(timetaken*1000))
    
    # plot both the lines and points to make it visually better
    plt.plot(solver.x, solver.psi_left, c='b',label=r'$\psi_\mathrm{left}$')
    plt.scatter(solver.x, solver.psi_left, c='b')
    
    plt.plot(solver.x, solver.psi_right, c='r',label=r'$\psi_\mathrm{right}$')
    plt.scatter(solver.x, solver.psi_right, c='r')
    plt.xlabel(r'$x$ (bohr)')
    plt.ylabel(r'$\psi(x)$')
    plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=53, description='npoints:', max=101, min=3, step=5), Output()), _dom_cla…

# Now let's write a better version of Numerov Solver where we normalize the wave funcitons

In [5]:
# An improved version that handles normalization
class NumerovSolverPIB_v2:
    def __init__(self, xlower, xupper, npoints=1000):
        self.xlower = xlower
        self.xupper = xupper
        self.npoints = npoints
        self.x = np.linspace(self.xlower,self.xupper,self.npoints)
        self.delta = self.x[1]-self.x[0]
        self.x1 = 1e-2 # A small positive value as the initial guess the psi value at the second grid point
        self.psi_left = None
        self.psi_right = None
        self.k2 = np.pi**2
        self.prob_left = None
        self.prob_right = None
    
    def propogateNumerov(self,x0,x1,psi0,psi1):
        psi2 = 2*psi1 - psi0 - self.k2*psi1*self.delta**2
        return psi2
    
    def Numerov_left(self):
        self.psi_left = np.zeros(len(self.x))
        self.psi_left[1] = self.x1
        for i in range(1,len(self.x)-1):
            self.psi_left[i+1] = self.propogateNumerov(self.x[i-1],self.x[i],self.psi_left[i-1],self.psi_left[i])
        self.prob_left = np.trapz(np.power(self.psi_left,2),self.x)
        self.psi_left = self.psi_left/np.sqrt(self.prob_left)
        
    def Numerov_right(self):
        self.psi_right = np.zeros(len(self.x))
        self.psi_right[-2] = self.x1
        for i in range(len(self.x)-2,0,-1):
            self.psi_right[i-1] = self.propogateNumerov(self.x[i+1],self.x[i],self.psi_right[i+1],self.psi_right[i])
        self.prob_right = np.trapz(np.power(self.psi_right,2),self.x)
        self.psi_right = self.psi_right/np.sqrt(self.prob_right)

In [6]:
fig3=plt.figure()
@widgets.interact(npoints=n_options)
def update3(npoints):
    plt.clf()
    solver=NumerovSolverPIB_v2(0,1,npoints)
    start=time.time()
    solver.Numerov_left()
    solver.Numerov_right()
    end=time.time()
    timetaken=end-start
    
    # The wave funciton is not normalized. 
    prob_left = np.trapz(np.power(solver.psi_left,2),solver.x)
    prob_right = np.trapz(np.power(solver.psi_right,2),solver.x)
    plt.title("Numerov solution to PIB, "
              + r"$\int\psi_\mathrm{left}$=" + "{:.2g}".format(prob_left)
              + r", $\int\psi_\mathrm{right}=$" + "{:.2g}".format(prob_right)
              + "\nTime taken: {:.2f} ms".format(timetaken*1000))
    
    # plot both the lines and points to make it visually better
    plt.plot(solver.x, solver.psi_left, c='b',label=r'$\psi_\mathrm{left}$')
    plt.scatter(solver.x, solver.psi_left, c='b')
    
    plt.plot(solver.x, solver.psi_right, c='r',label=r'$\psi_\mathrm{right}$')
    plt.scatter(solver.x, solver.psi_right, c='r')
    
    # Plot the reference analytical solution
    x = np.linspace(0,1,1000)
    psi = np.sqrt(2)*np.sin(np.pi*x)
    plt.plot(x, psi, c='k',label=r'$\psi$')
    
    plt.xlabel(r'$x$ (bohr)')
    plt.ylabel(r'$\psi(x)$')
    plt.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=18, description='npoints:', max=101, min=3, step=5), Output()), _dom_cla…